<a href="https://colab.research.google.com/github/Leo777Deo/CFD_Project/blob/main/Different_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#CASE 1:
# Works but fan at end, facciamo respirare il boundary con thick_FF
from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 5 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.005 # Additive quantity to the end point of the convergent to close it
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump - Delta_conv], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up

# Define Far Field
Y_DIST = 150 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -20 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = -X_DIST * x_vals[np.argmax(x_vals)]


size_bl = 0.00001
n_bl = 65 # Number of layers  # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("Bumpconv.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

# Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

# Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1

    # Write the convex shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 1}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 2}) = {{{upper_point_start}, {upper_point_end}}};\n")

    # Define connecting lines for closed convex surface
    file.write(f"Line({len(x_vals) + 3}) = {{{new_point_start}, {upper_point_start}}};\n")
    file.write(f"Line({len(x_vals) + 4}) = {{{new_point_end}, {upper_point_end}}};\n")

    # Boundary points for the entire domain
    boundary_point_1 = upper_point_end + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1 # intermediate inlet
    boundary_point_7 = boundary_point_6 + 1 # intermediate outlet

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_5}) = {{{X_START}, 0, 0, H}};\n")

    file.write(f"Point({boundary_point_6}) = {{{X_START}, {thick_FF}, 0, H}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {thick_FF}, 0, H}};\n") # thick_bl

    # Define lines for the main boundary
    file.write(f"Line({len(x_vals) + 6}) = {{{boundary_point_1}, {boundary_point_2}}};\n")
    file.write(f"Line({len(x_vals) + 7}) = {{{boundary_point_7}, {boundary_point_3}}};\n") # to chang
    file.write(f"Line({len(x_vals) + 8}) = {{{boundary_point_3}, {boundary_point_4}")
    file.write("};\n")
    file.write(f"Line({len(x_vals) + 9}) = {{{boundary_point_4}, {boundary_point_6}}};\n") # to change
    file.write(f"Line({len(x_vals) + 10}) = {{{boundary_point_5}, 1}};\n")  # Connect back to start of bump
    file.write(f"Line({len(x_vals) + 11}) = {{{new_point_start - 1},{boundary_point_1} }};\n")

    file.write(f"Line({len(x_vals) + 12}) = {{{boundary_point_6},{boundary_point_5} }};\n")
    file.write(f"Line({len(x_vals) + 13}) = {{{boundary_point_2},{boundary_point_7} }};\n")

    file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 13}}} = {n_bl} Using Progression {r};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {")
    file.write("1, 2") # Spline 1 and 2
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6},{len(x_vals) + 13} ,{len(x_vals) + 7}, {len(x_vals) + 8}, {len(x_vals) + 9},{len(x_vals) + 12} ,{len(x_vals) + 10}")
    file.write("};\n")

    file.write("Line Loop(2) = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    file.write("};\n")

    file.write("Plane Surface(1) = {1, 2};\n") # It tells GMSH to mesh only on 1 and not account for 2

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{len(x_vals) + 9},{len(x_vals) + 12} }};\n")

    file.write("Physical Line('lower wall') = {")
    file.write(f"{len(x_vals) + 10},")
    file.write("1, 2")
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}")
    file.write("};\n")

   # file.write(f"Physical Line('upper wall') = {{{len(x_vals) + 8}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    file.write("};\n")

    file.write(f"Physical Line('outlet') = {{{len(x_vals) + 13},{len(x_vals) + 7}, {len(x_vals) + 8}}};\n")


    # MESH:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {")
    file.write(f"{len(x_vals) + 10},")
    file.write("1, 2")
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}")
    file.write("};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    #file.write("Field[2] = BoundaryLayer;\n")
    #file.write("Field[2].CurvesList = {")
    #file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    #file.write("};\n")
    #file.write("Field[2].Quads = 1;\n")
    #file.write(f"Field[2].Ratio = {r};\n")
    #file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    #file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    #file.write("Field[2].FanPointsList = {1};\n")
    #file.write("Field[2].FanPointsSizesList = {5};\n")
    #file.write("BoundaryLayer Field = 2;\n")

from google.colab import files
files.download("Bumpconv.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
#CASE 1.5: Circle and transfinite at end domain points
# Works but fan at end, facciamo respirare il boundary con thick_FF
from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 5 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.005 # Additive quantity to the end point of the convergent to close it
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

#x_start_up, y_start_up = x_vals[index_line_bump - Delta_conv], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
#y_end_up = y_start_up

# Define Far Field
Y_DIST = 150 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -20 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = -X_DIST * x_vals[np.argmax(x_vals)]

DIAM = 0.01 # Diameter of circle at tip of convergent
y_end_up = y_start + DIAM

size_bl = 0.00001
n_bl = 65 # Number of layers  # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("Bumpconv.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write("h_conv = 0.001;\n") # Size of cells of Convergent
    #file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

# Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

# Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1

    # Write the convex shape points and lines 1->4->-2
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h_conv}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h_conv}};\n")
    file.write(f"Point({upper_point_start}) = {{{x_start}, {y_end_up}, 0, h_conv}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h_conv}};\n")

    file.write(f"Line({len(x_vals) + 1}) = {{{new_point_start}, {new_point_end}}};\n")
    file.write(f"Line({len(x_vals) + 2}) = {{{upper_point_start}, {upper_point_end}}};\n")
    file.write(f"Line({len(x_vals) + 4}) = {{{new_point_end}, {upper_point_end}}};\n")

    # Boundary points for the entire domain
    boundary_point_1 = upper_point_end + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1 # intermediate inlet
    boundary_point_7 = boundary_point_6 + 1 # intermediate outlet
    boundary_point_8 = boundary_point_7 + 1 # intermediate START
    boundary_point_9 = boundary_point_8 + 1 # intermediate END
    circle_centre = boundary_point_9+ 1 # Circle centre

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_5}) = {{{X_START}, 0, 0, H}};\n")

    file.write(f"Point({boundary_point_6}) = {{{X_START}, {thick_FF}, 0, H}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {thick_FF}, 0, H}};\n") # thick_bl

    file.write(f"Point({boundary_point_8}) = {{{X_START + thick_FF }, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_9}) = {{{X_END - thick_FF}, 0, 0, H}};\n")

    # Circle definition at tip of convergent:
    file.write(f"Point({circle_centre}) = {{{(DIAM/2)+x_vals[index_line_bump]}, {(y_start + y_end_up)/2}, 0, h_conv}};\n")
    file.write(f"Circle({len(x_vals) + 16}) = {{{new_point_start}, {circle_centre}, {upper_point_start}}};\n")  # Circle definition (start point, centre point, end point)

    # Define lines for the main boundary
    file.write(f"Line({len(x_vals) + 6}) = {{{boundary_point_1}, {boundary_point_9}}};\n") #Changed:  Connect back to END
    file.write(f"Line({len(x_vals) + 7}) = {{{boundary_point_7}, {boundary_point_3}}};\n") #changed
    file.write(f"Line({len(x_vals) + 8}) = {{{boundary_point_3}, {boundary_point_4}")
    file.write("};\n")
    file.write(f"Line({len(x_vals) + 9}) = {{{boundary_point_4}, {boundary_point_6}}};\n") # changed
    file.write(f"Line({len(x_vals) + 10}) = {{{boundary_point_8}, 1}};\n")  #Changed:  Connect back to start of bump
    file.write(f"Line({len(x_vals) + 11}) = {{{new_point_start - 1},{boundary_point_1} }};\n")

    file.write(f"Line({len(x_vals) + 12}) = {{{boundary_point_6},{boundary_point_5} }};\n")
    file.write(f"Line({len(x_vals) + 13}) = {{{boundary_point_2},{boundary_point_7} }};\n")

    file.write(f"Line({len(x_vals) + 14}) = {{{boundary_point_5},{boundary_point_8} }};\n")
    file.write(f"Line({len(x_vals) + 15}) = {{{boundary_point_9},{boundary_point_2} }};\n")

    file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 13}}} = {n_bl} Using Progression {r};\n")

    file.write(f"Transfinite Line{{{len(x_vals) + 14}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 15}}} = {n_bl} Using Progression {r};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {")
    file.write("1, 2") # Spline 1 and 2
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6},{len(x_vals) + 15},{len(x_vals) + 13} ,{len(x_vals) + 7}, {len(x_vals) + 8}, {len(x_vals) + 9},{len(x_vals) + 12} ,{len(x_vals) + 14},{len(x_vals) + 10}")
    file.write("};\n")

    file.write("Line Loop(2) = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals)+16}")
    file.write("};\n")

    file.write("Plane Surface(1) = {1, 2};\n") # It tells GMSH to mesh only on 1 and not account for 2

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{len(x_vals) + 9},{len(x_vals) + 12} }};\n")

    file.write("Physical Line('lower wall') = {")
    file.write(f"{len(x_vals) + 14},{len(x_vals) + 10},")
    file.write("1, 2")
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6},{len(x_vals) + 15}")
    file.write("};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2},-{len(x_vals)+16}")
    file.write("};\n")

    file.write(f"Physical Line('outlet') = {{{len(x_vals) + 13},{len(x_vals) + 7}, {len(x_vals) + 8}}};\n")


    # MESH:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {")
    file.write(f"{len(x_vals) + 10},")
    file.write("1, 2")
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}")
    file.write("};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    #file.write("Field[2] = BoundaryLayer;\n")
    #file.write("Field[2].CurvesList = {")
    #file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2},-{len(x_vals)+16}")
    #file.write(f"{len(x_vals) + 1}")
    #file.write("};\n")
    #file.write("Field[2].Quads = 1;\n")
    #file.write(f"Field[2].Ratio = {r};\n")
    #file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    #file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    #file.write("Field[2].FanPointsList = {1};\n")
    #file.write("Field[2].FanPointsSizesList = {5};\n")
    #file.write("BoundaryLayer Field = 2;\n")

from google.colab import files
files.download("Bumpconv.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
#CASE 2: Only Convergent
from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 5 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.005 # Additive quantity to the end point of the convergent to close it
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

#x_start_up, y_start_up = x_vals[index_line_bump - Delta_conv], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
#y_end_up = y_start_up

# Define Far Field
Y_DIST = 150 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -20 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = -X_DIST * x_vals[np.argmax(x_vals)]

DIAM = 0.01 # Diameter of circle at tip of convergent
y_end_up = y_start + DIAM

size_bl = 0.00001
n_bl = 65 # Number of layers  # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("CONV.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write("h = 0.01;\n") # Size of cells close to bump
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write("h_conv = 0.001;\n") # Size of cells of Convergent

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1

    # Write the convex shape points and lines 1->4->-2
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h_conv}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h_conv}};\n")
    file.write(f"Point({upper_point_start}) = {{{x_start}, {y_end_up}, 0, h_conv}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h_conv}};\n")

    file.write(f"Line({1}) = {{{new_point_start}, {new_point_end}}};\n")
    file.write(f"Line({2}) = {{{new_point_end}, {upper_point_end}}};\n")
    file.write(f"Line({3}) = {{{upper_point_start}, {upper_point_end}}};\n")

    # Boundary points for the entire domain
    boundary_point_1 = upper_point_end + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1 # intermediate outlet
    boundary_point_8 = boundary_point_7 + 1 # intermediate START
    boundary_point_9 = boundary_point_8 + 1 # intermediate END
    boundary_point_10 = boundary_point_9 + 1 # intermediate START
    circle_centre = boundary_point_10+ 1 # Circle centre

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_5}) = {{{X_START}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_6}) = {{{x_start},0, 0, h}};\n")
    file.write(f"Point({boundary_point_7}) = {{{x_end},{Y_END}, 0, h}};\n")
    file.write(f"Point({boundary_point_8}) = {{{x_start},{Y_END}, 0, h}};\n")
    file.write(f"Point({boundary_point_9}) = {{{X_START},{y_end_up}, 0, h}};\n")
    file.write(f"Point({boundary_point_10}) = {{{X_START},{y_start}, 0, h}};\n")

    # Circle definition at tip of convergent:
    file.write(f"Point({circle_centre}) = {{{(DIAM/2)+x_vals[index_line_bump]}, {(y_start + y_end_up)/2}, 0, h_conv}};\n")
    file.write(f"Circle({4}) = {{{new_point_start}, {circle_centre}, {upper_point_start}}};\n")  # Circle definition (start point, centre point, end point)

    # Define lines for the main boundary
    file.write(f"Line({5}) = {{{boundary_point_1}, {boundary_point_2}}};\n")
    file.write(f"Line({6}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({7}) = {{{boundary_point_3}, {boundary_point_7}")
    file.write("};\n")
    file.write(f"Line({8}) = {{{boundary_point_7}, {boundary_point_8}}};\n")
    file.write(f"Line({9}) = {{{boundary_point_8}, {boundary_point_4}}};\n")
    file.write(f"Line({10}) = {{{boundary_point_4},{boundary_point_9} }};\n")
    file.write(f"Line({11}) = {{{boundary_point_9},{boundary_point_10} }};\n")
    file.write(f"Line({12}) = {{{boundary_point_10},{boundary_point_5} }};\n")
    file.write(f"Line({13}) = {{{boundary_point_5},{boundary_point_6} }};\n")
    file.write(f"Line({14}) = {{{boundary_point_6},{boundary_point_1} }};\n")
    #file.write(f"Line({15}) = {{{upper_point_start},{boundary_point_9} }};\n")
    file.write(f"Line({16}) = {{{new_point_start},{boundary_point_10} }};\n")
    file.write(f"Line({17}) = {{{boundary_point_1},{new_point_end} }};\n")
    file.write(f"Line({18}) = {{{upper_point_end},{boundary_point_7} }};\n")
    file.write(f"Line({19}) = {{{boundary_point_6},{new_point_start} }};\n")

    file.write(f"Transfinite Line(1) = 60 Using Progression {r};\n")
    file.write(f"Transfinite Line(14) = 60 Using Progression {r};\n")

    file.write(f"Transfinite Line(5) = 60 Using Progression {r};\n")
    file.write(f"Transfinite Line(7) = 60 Using Progression {1/r};\n")

    file.write(f"Transfinite Line(8) = 60 Using Progression {1/r};\n")
    file.write(f"Transfinite Line(3) = 60 Using Progression {r};\n")

    file.write(f"Transfinite Line(9) = 60 Using Progression {1/r};\n")
    file.write(f"Transfinite Line(16) = 60 Using Progression {r};\n")
    file.write(f"Transfinite Line(13) = 60 Using Progression {r};\n")

    file.write(f"Transfinite Line(10) = 30 Using Progression {1/r};\n")
    file.write(f"Transfinite Line(12) = 30 Using Progression {1/r};\n")
    file.write(f"Transfinite Line(6) = 60 Using Progression {r};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {") # CONVERGENT
    file.write(f"-1,4,3,-2")
    file.write("};\n")

    file.write("Line Loop(2) = {") # Below convergent
    file.write(f"14,17,-1,-19")
    file.write("};\n")

    file.write("Line Loop(3) = {") # Front of convergent
    file.write("12,13,19,16")
    file.write("};\n")

    file.write("Line Loop(4) = {") # Domain
    file.write("5,6,7,8,9,10,11,12,13,14")
    file.write("};\n")


    file.write("Plane Surface(1) = {1};\n")
    file.write("Plane Surface(2) = {2};\n")
    file.write("Plane Surface(3) = {3};\n")
    file.write("Plane Surface(4) = {4};\n")

    file.write("Transfinite Surface{2} = {")
    file.write(f"{boundary_point_6},{boundary_point_1},{new_point_end},{new_point_start}") # Points that make up domain
    file.write("};\n")
    file.write("Recombine Surface{2};\n")

    file.write("Transfinite Surface{3} = {")
    file.write(f"{boundary_point_5},{boundary_point_6},{new_point_start},{boundary_point_10}") # Points that make up domain
    file.write("};\n")
    file.write("Recombine Surface{3};\n")

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {4,1};\n")

    # Physical lines and surfaces for boundary conditions
    file.write("Physical Line('inlet') = {")
    file.write(f"10,11,12")
    file.write("};\n")

    file.write("Physical Line('lower wall') = {")
    file.write("13,14,5")
    file.write("};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"-1,4,3,-2")
    file.write("};\n")

    file.write("Physical Line('outlet') = {")
    file.write(f"6,7,8,9")
    file.write("};\n")


from google.colab import files
files.download("CONV.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
#CASE 2.5: Only Convergent, no structored
from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 5 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.005 # Additive quantity to the end point of the convergent to close it
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

#x_start_up, y_start_up = x_vals[index_line_bump - Delta_conv], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
#y_end_up = y_start_up

# Define Far Field
Y_DIST = 150 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -20 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = -X_DIST * x_vals[np.argmax(x_vals)]

DIAM = 0.01 # Diameter of circle at tip of convergent
y_end_up = y_start + DIAM

size_bl = 0.00001
n_bl = 65 # Number of layers  # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("CONV.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write("h = 0.1;\n") # Size of cells close to bump
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write("h_conv = 0.05;\n") # Size of cells of Convergent

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1

    # Write the convex shape points and lines 1->4->-2
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h_conv}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h_conv}};\n")
    file.write(f"Point({upper_point_start}) = {{{x_start}, {y_end_up}, 0, h_conv}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h_conv}};\n")

    file.write(f"Line({1}) = {{{new_point_start}, {new_point_end}}};\n")
    file.write(f"Line({2}) = {{{new_point_end}, {upper_point_end}}};\n")
    file.write(f"Line({3}) = {{{upper_point_start}, {upper_point_end}}};\n")

    # Boundary points for the entire domain
    boundary_point_1 = upper_point_end + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1 # intermediate outlet
    boundary_point_8 = boundary_point_7 + 1 # intermediate START
    boundary_point_9 = boundary_point_8 + 1 # intermediate END
    boundary_point_10 = boundary_point_9 + 1 # intermediate START
    circle_centre = boundary_point_10+ 1 # Circle centre

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h_conv}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_5}) = {{{X_START}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_6}) = {{{x_start},0, 0, h_conv}};\n")
    file.write(f"Point({boundary_point_7}) = {{{x_end},{Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_8}) = {{{x_start},{Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_9}) = {{{X_START},{y_end_up}, 0, H}};\n")
    file.write(f"Point({boundary_point_10}) = {{{X_START},{y_start}, 0,H}};\n")

    # Circle definition at tip of convergent:
    file.write(f"Point({circle_centre}) = {{{(DIAM/2)+x_vals[index_line_bump]}, {(y_start + y_end_up)/2}, 0, h_conv}};\n")
    file.write(f"Circle({4}) = {{{new_point_start}, {circle_centre}, {upper_point_start}}};\n")  # Circle definition (start point, centre point, end point)

    # Define lines for the main boundary
    file.write(f"Line({5}) = {{{boundary_point_1}, {boundary_point_2}}};\n")
    file.write(f"Line({6}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({7}) = {{{boundary_point_3}, {boundary_point_7}")
    file.write("};\n")
    file.write(f"Line({8}) = {{{boundary_point_7}, {boundary_point_8}}};\n")
    file.write(f"Line({9}) = {{{boundary_point_8}, {boundary_point_4}}};\n")
    file.write(f"Line({10}) = {{{boundary_point_4},{boundary_point_9} }};\n")
    file.write(f"Line({11}) = {{{boundary_point_9},{boundary_point_10} }};\n")
    file.write(f"Line({12}) = {{{boundary_point_10},{boundary_point_5} }};\n")
    file.write(f"Line({13}) = {{{boundary_point_5},{boundary_point_6} }};\n")
    file.write(f"Line({14}) = {{{boundary_point_6},{boundary_point_1} }};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {") # CONVERGENT
    file.write(f"-1,4,3,-2")
    file.write("};\n")

    file.write("Line Loop(4) = {") # Domain
    file.write("5,6,7,8,9,10,11,12,13,14")
    file.write("};\n")

    file.write("Plane Surface(1) = {4,1};\n")

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1};\n")

    # Physical lines and surfaces for boundary conditions
    file.write("Physical Line('inlet') = {")
    file.write(f"10,11,12")
    file.write("};\n")

    file.write("Physical Line('lower wall') = {")
    file.write("13,14,5")
    file.write("};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"-1,4,3,-2")
    file.write("};\n")

    file.write("Physical Line('outlet') = {")
    file.write(f"6,7,8,9")
    file.write("};\n")


from google.colab import files
files.download("CONV.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# CASE 3:

# Make all inlet and outlet as transfinite lines

from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 5 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.005 # Additive quantity to the end point of the convergent to close it
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump - Delta_conv], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up

# Define Far Field
Y_DIST = 150 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -25 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = 25 * x_vals[np.argmax(x_vals)]


size_bl = 0.00001
n_bl = 65 # Number of layers  # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl

# .geo file creation:
with open("Bumpconv.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

# Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

# Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1

    # Write the convex shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 1}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 2}) = {{{upper_point_start}, {upper_point_end}}};\n")

    # Define connecting lines for closed convex surface
    file.write(f"Line({len(x_vals) + 3}) = {{{new_point_start}, {upper_point_start}}};\n")
    file.write(f"Line({len(x_vals) + 4}) = {{{new_point_end}, {upper_point_end}}};\n")

    # Boundary points for the entire domain
    boundary_point_1 = upper_point_end + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1 # intermediate inlet
    boundary_point_7 = boundary_point_6 + 1 # intermediate outlet

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_5}) = {{{X_START}, 0, 0, H}};\n")

    file.write(f"Point({boundary_point_6}) = {{{X_START}, {thick_FF}, 0, H}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {thick_FF}, 0, H}};\n") # thick_bl

    # Define lines for the main boundary
    file.write(f"Line({len(x_vals) + 6}) = {{{boundary_point_1}, {boundary_point_2}}};\n")
    file.write(f"Line({len(x_vals) + 7}) = {{{boundary_point_7}, {boundary_point_3}}};\n") # to chang
    file.write(f"Line({len(x_vals) + 8}) = {{{boundary_point_3}, {boundary_point_4}")
    file.write("};\n")
    file.write(f"Line({len(x_vals) + 9}) = {{{boundary_point_4}, {boundary_point_6}}};\n") # to change
    file.write(f"Line({len(x_vals) + 10}) = {{{boundary_point_5}, 1}};\n")  # Connect back to start of bump
    file.write(f"Line({len(x_vals) + 11}) = {{{new_point_start - 1},{boundary_point_1} }};\n")

    file.write(f"Line({len(x_vals) + 12}) = {{{boundary_point_6},{boundary_point_5} }};\n")
    file.write(f"Line({len(x_vals) + 13}) = {{{boundary_point_2},{boundary_point_7} }};\n")

    file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 13}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 9}}} = {150} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 7}}} = {150} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 10}}} = {500} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 6}}} = {500} Using Progression {1};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {")
    file.write("1, 2") # Spline 1 and 2
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6},{len(x_vals) + 13} ,{len(x_vals) + 7}, {len(x_vals) + 8}, {len(x_vals) + 9},{len(x_vals) + 12} ,{len(x_vals) + 10}")
    file.write("};\n")

    file.write("Line Loop(2) = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    file.write("};\n")

    file.write("Plane Surface(1) = {1, 2};\n") # It tells GMSH to mesh only on 1 and not account for 2

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{len(x_vals) + 9},{len(x_vals) + 12} }};\n")

    file.write("Physical Line('lower wall') = {")
    file.write(f"{len(x_vals) + 10},")
    file.write("1, 2")
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}")
    file.write("};\n")

    file.write(f"Physical Line('upper wall') = {{{len(x_vals) + 8}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    file.write("};\n")

    file.write(f"Physical Line('outlet') = {{{len(x_vals) + 13},{len(x_vals) + 7}}};\n")


    # MESH:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {")
    file.write(f"{len(x_vals) + 10},")
    file.write("1, 2")
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}")
    file.write("};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

from google.colab import files
files.download("Bumpconv.geo")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#CASE 4:
# Can be runned, all unstructured

from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 5 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.005 # Additive quantity to the end point of the convergent to close it
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump - Delta_conv], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up

# Define Far Field
Y_DIST = 150 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -25 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = 25 * x_vals[np.argmax(x_vals)]

# .geo file creation:
with open("geometry.geo", "w") as file:

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1.5;\n") # Size of cells close to farfield
    file.write("r = 1.0;\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

# Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

# Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1

    # Write the convex shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 1}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 2}) = {{{upper_point_start}, {upper_point_end}}};\n")

    # Define connecting lines for closed convex surface
    file.write(f"Line({len(x_vals) + 3}) = {{{new_point_start}, {upper_point_start}}};\n")
    file.write(f"Line({len(x_vals) + 4}) = {{{new_point_end}, {upper_point_end}}};\n")

    # Boundary points for the entire domain
    boundary_point_1 = upper_point_end + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, 0, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_5}) = {{{X_START}, 0, 0, H}};\n")

    # Define lines for the main boundary
    file.write(f"Line({len(x_vals) + 6}) = {{{boundary_point_1}, {boundary_point_2}}};\n")
    file.write(f"Line({len(x_vals) + 7}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({len(x_vals) + 8}) = {{{boundary_point_3}, {boundary_point_4}")
    file.write("};\n")
    file.write(f"Line({len(x_vals) + 9}) = {{{boundary_point_4}, {boundary_point_5}}};\n")
    file.write(f"Line({len(x_vals) + 10}) = {{{boundary_point_5}, 1}};\n")  # Connect back to start of bump
    file.write(f"Line({len(x_vals) + 11}) = {{{new_point_start - 1},{boundary_point_1} }};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {")
    file.write("1, 2") # Spline 1 and 2
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}, {len(x_vals) + 7}, {len(x_vals) + 8}, {len(x_vals) + 9}, {len(x_vals) + 10}")
    file.write("};\n")

    file.write("Line Loop(2) = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    file.write("};\n")

    file.write("Plane Surface(1) = {1, 2};\n") # It tells GMSH to mesh only on 1 and not account for 2

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{len(x_vals) + 9}}};\n")

    file.write("Physical Line('lower wall') = {")
    file.write(f"{len(x_vals) + 10},")
    file.write("1, 2") # Spline 1 and 2
    file.write(f",{len(x_vals) + 11}, {len(x_vals) + 6}")
    file.write("};\n")

    file.write(f"Physical Line('upper wall') = {{{len(x_vals) + 8}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{len(x_vals) + 1}, {len(x_vals) + 4}, -{len(x_vals) + 2}, -{len(x_vals) + 3}")
    file.write("};\n")

    file.write(f"Physical Line('outlet') = {{{len(x_vals) + 7}}};\n")


    # MESH:

# 1: MeshAdapt, 2: Automatic, 3: Initial mesh only, 5: Delaunay, 6: Frontal-Delaunay,
# 7: BAMG, 8: Frontal-Delaunay for Quads, 9: Packing of Parallelograms

    file.write("Mesh.Algorithm = 6; // Frontal-Delaunay for quads\n")
    file.write("Mesh.RecombineAll = 1; // Enable recombination for quad elements\n")

    # Download the file (if running in an environment that supports file download)
from google.colab import files
files.download("geometry.geo")